# 그냥 한번 뉴럴네트웤 테스트...ㅎ
* 데이터는 아래 데이터와, 그리고 각 사람당 리뷰한 맥주 데이터의 평균을 임베딩하여 추가함


In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

### 데이터 설명
* review 테이블 : 사용자가 맥주에 준 점수, 즉 y값 하나만을 위해 가져옴
* beer 테이블 : 맥주 데이터
* member 테이블 : 멤버 나이, 성별
* member_mean 테이블 : 사용자의 현재까지 기록의 평균

In [2]:
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = '''SELECT b.abv AS beer_abv, b.brewer_id, b.country, b.large_category, b.sub_category, m.age, m.gender, ms.*, r.overall AS y
    FROM review AS r
    INNER JOIN beer AS b ON r.beer_id = b.id
    INNER JOIN member AS m ON r.member_id = m.id
    INNER JOIN member_sum AS ms ON r.member_id = ms.member_id
    WHERE b.id IN (1, 2, 3);'''
reviews = sqlio.read_sql_query(sql, conn)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_10516\142036897.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews = sqlio.read_sql_query(sql, conn)


In [3]:
# 불러온 데이터, age랑 gender는 못써먹어서 일단 제외
reviews = reviews.drop(columns=['member_id', 'age', 'gender'])

In [4]:
# 나누는것은 파이썬에서 해야하나...?ㅠㅠ
review_notdiv = reviews[['beer_abv', 'brewer_id', 'country', 'large_category', 'sub_category', 'y']]
review_todiv = reviews.drop(columns=['beer_abv', 'brewer_id', 'country', 'large_category', 'sub_category', 'y'])

In [5]:
review_todiv = review_todiv.div(review_todiv['divide_size'], axis=0)
review_todiv = review_todiv.drop(columns='divide_size')

In [6]:
merged = pd.concat([review_notdiv, review_todiv], axis=1)

In [8]:
merged

,beer_abv,brewer_id,country,large_category,sub_category,y,overall,abv,review_count,country_AR,...,sub_category_Specialty Grain,sub_category_Spice/Herb/Vegetable,sub_category_Stout,sub_category_Strong Pale Lager/Imperial Pils,sub_category_Sweet Stout,sub_category_Traditional Ale,sub_category_Vienna,sub_category_Weizen Bock,sub_category_Wheat Ale,sub_category_Zwickel/Keller/Landbier
0,5.0,1,US,Lagers,Vienna,11,14.100873,6.503337,379.641125,0.000000,...,0.004365,0.022793,0.026673,0.004365,0.015519,0.006305,0.004850,0.003880,0.023763,0.001455
1,5.0,1,US,Lagers,Vienna,7,12.764771,6.780591,710.807135,0.000000,...,0.003344,0.025641,0.024526,0.004459,0.017837,0.003344,0.006689,0.007804,0.023411,0.002230
2,5.0,1,US,Lagers,Vienna,13,14.493952,6.382460,657.821573,0.000000,...,0.010081,0.020161,0.023185,0.006048,0.018145,0.003024,0.009073,0.004032,0.015121,0.004032
3,5.0,1,US,Lagers,Vienna,5,12.826682,6.362189,486.534208,0.000000,...,0.005131,0.031357,0.025656,0.004561,0.026226,0.004561,0.009122,0.003991,0.018244,0.001710
4,5.0,1,US,Lagers,Vienna,11,13.300000,5.557417,1255.916667,0.000000,...,0.008333,0.000000,0.050000,0.000000,0.000000,0.008333,0.016667,0.008333,0.041667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,4.1,184,IE,Stout and Porter,Dry Stout,12,12.851613,5.567839,848.832258,0.000000,...,0.003226,0.016129,0.019355,0.003226,0.000000,0.006452,0.022581,0.000000,0.032258,0.000000
2046,4.1,184,IE,Stout and Porter,Dry Stout,11,10.950879,5.898279,273.698062,0.000451,...,0.007661,0.024335,0.012618,0.018927,0.004957,0.009914,0.028842,0.003155,0.004507,0.013520
2047,4.1,184,IE,Stout and Porter,Dry Stout,17,14.000000,4.906364,1353.272727,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000
2048,4.1,184,IE,Stout and Porter,Dry Stout,10,12.728548,6.020988,399.174312,0.000540,...,0.006476,0.022126,0.024825,0.004317,0.019968,0.003238,0.009714,0.003778,0.019428,0.002159


## 전처리
* 가져온 데이터 전처리 함
* 일단 무식하게 다 바꾸는 식으로 했는데
  * 이제 beer_id에 따라 범주화 후 그 데이터를 사용자리뷰와 join시키는 식으로 활용해야 할 듯

In [9]:
from sklearn.preprocessing import OneHotEncoder

# 인코더 설정, none값은 그냥 0 박음
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 숫자형 데이터
numeric_data = merged.drop(columns=['brewer_id', 'large_category', 'sub_category', 'country'])

# 범주형 데이터 맞추고 변환
encoder.fit(merged[['brewer_id', 'large_category', 'sub_category', 'country']])
encoding_beer = encoder.transform(merged[['brewer_id', 'large_category', 'sub_category', 'country']])
# # 다시 pandas 데이터프레임으로 변환해서 붙일수 있도록 만듬
encoded_categories_beer = pd.DataFrame(encoding_beer, columns=encoder.get_feature_names_out(input_features=['brewer_id', 'large_category', 'sub_category', 'country']))
# print(processed_beer_notscaled)
merged = pd.concat([numeric_data, encoded_categories_beer], axis=1)

In [12]:
y = merged['y']
X = merged.drop(columns=['y'])

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

## 전처리가 모두 끝나면 regressor로 예측함

In [20]:
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(max_iter=500, hidden_layer_sizes=(100,50,50)).fit(X, y)

In [25]:
regr.predict(X[:100])

array([10.586924  ,  6.86896129, 12.74239003,  5.16976601, 10.95118474,
       10.66497495, 13.24401414, 10.77287792, 13.524316  ,  9.44766582,
        6.16387217,  5.91863495,  5.86455249,  9.47493961, 10.80756707,
       10.73137087,  3.76769783, 12.92760861, 14.14343351,  3.60790765,
        9.37051127, 16.97474077,  8.71227469,  9.85868947, 12.29719964,
        8.59046126,  6.23616363, 11.63556657, 10.63620005, 12.56332472,
       13.16570231, 14.31542256,  9.68217333, 12.27707181,  5.80784086,
        7.62603587,  9.7925704 , 13.83350399,  8.62648129,  9.32580664,
       11.24330489, 11.72755193,  7.49291251,  5.75293283,  9.62247956,
       12.44748796, 10.55644059, 12.48691971, 11.86119827, 12.1214814 ,
        9.28520002, 14.67684893, 11.88054916, 10.82635292, 12.64814201,
       11.89187218, 10.91623448, 12.68020524, 11.01789628, 11.45136076,
       11.44849821,  9.36038769,  8.99270663, 11.7535846 ,  9.43011449,
       15.68307911, 10.34850605,  8.84345327,  6.84707401,  6.67

In [22]:
y[:10]

0    11
1     7
2    13
3     5
4    11
5    11
6    13
7    11
8    14
9    10
Name: y, dtype: int64